In [1]:
import pandas as pd
import numpy as np
import datetime

# Load Reddit comments

In [2]:
li = []
df1 = pd.read_csv('bitcoinsubreddit01.csv')
li.append(df1)
df2 = pd.read_csv('bitcoinsubreddit02.csv')
li.append(df2)
df3 = pd.read_csv('bitcoinsubreddit03.csv')
li.append(df3)
df4 = pd.read_csv('bitcoinsubreddit04.csv')
li.append(df4)
df5 = pd.read_csv('bitcoinsubreddit05.csv')
li.append(df5)
df6 = pd.read_csv('bitcoinsubreddit06.csv')
li.append(df6)

df = pd.concat(li, axis=0, ignore_index=True)

In [3]:
df["created_utc"] = pd.to_datetime(df["created_utc"],unit='s')

In [4]:
df["retrieved_on"] = pd.to_datetime(df["retrieved_on"],unit='s')

# Load BitCoin price history

In [6]:
dfprice = pd.read_csv('BitCoinPrice.csv')

In [7]:
dfprice.dtypes

Date        object
Price       object
Open        object
High        object
Low         object
Vol.        object
Change %    object
dtype: object

In [8]:
dfprice["Date"] = pd.to_datetime(dfprice["Date"], infer_datetime_format=True)  

# Filter comments to just June and do necessary transformations

In [37]:
dfjune = df[df["created_utc"] >= '06/01/2021']

In [38]:
dfjune = dfjune[dfjune["created_utc"] < '07/01/2021']

In [39]:
dfjune["created_utc"] = pd.to_datetime(dfjune["created_utc"].dt.date)

In [48]:
dfjune["author_premium"] = dfjune["author_premium"].astype(int)

In [60]:
dfjune["locked"] = dfjune["locked"].astype(int)

In [65]:
dfjune["no_follow"] = dfjune["no_follow"].astype(int)

In [70]:
dfjune["quarantined"] = dfjune["quarantined"].astype(int)

# Filter BitCoin price data to just June

In [41]:
dfpricejune = dfprice[dfprice["Date"] >= '06/01/2021']

In [42]:
dfpricejune = dfpricejune[dfpricejune["Date"] < '07/01/2021']

# Filter out invalid comments

In [122]:
dfjunevalidcomments = dfjune[dfjune['body'] != "[removed]"]

In [123]:
dfjunevalidcomments = dfjunevalidcomments[dfjunevalidcomments['body'] != "[deleted]"]

# Get comment sentiment for each day in June
## First get a sampleof 165 comments from each day

In [124]:
currentday = datetime.datetime(2021,6,1)

li = []
num = 30
for x in range(num):
    dfjuneday = dfjunevalidcomments[dfjunevalidcomments["created_utc"] == currentday].sample(n=165)
    li.append(dfjuneday)
    currentday = currentday + datetime.timedelta(days=1)
    
dfjunesample = pd.concat(li, axis=0, ignore_index=True)

## Trim comments to 5,120 characters

The service being used to get comment sentiment can only accept strings up to 5,120 characters so we need to trim any comments longer than that.

In [3]:
dfjunesample["body"] = dfjunesample["body"].str.slice(0,5120)

## Set the sentiment field to 'Not Set'

Sometimes getting sentiments has to be done in batches, this allows us to track which comments we have already gotten the sentiment for.

In [ ]:
dfjunesample["sentiment"] = "Not Set"

## Setup method for getting sentiment

In [5]:
key = "9fbe75db05be4f4ebdfb176417a72bce"

In [6]:
endpoint = "https://masterssentiment.cognitiveservices.azure.com/"

In [7]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

## Get the sentiment for the comments

In [42]:
dfjunesample.to_csv("junesamplesentiment.csv")

In [2]:
dfjunesample = pd.read_csv("junesamplesentiment.csv")

In [14]:
dfjunesample["body"] = dfjunesample["body"].str.slice(0,5120)

In [10]:
dfjunesample["bodylength"] = dfjunesample["body"].str.len()

In [11]:
dfjunesample[dfjunesample["bodylength"] > 5120]

,Unnamed: 0,author_premium,body,controversiality,created_utc,distinguished,id,locked,no_follow,parent_id,quarantined,removal_reason,retrieved_on,score,subreddit,sentiment,bodylength
3793,3793,0,"How crazy you ask?\n\nOn April 30, 2012, McAfe...",0,2021-06-23,NaN,h2t27ua,0,1,t1_h2t04pz,0,NaN,1970-01-01,6,Bitcoin,Not Set,5526
4518,4518,0,I think this time will be different. Bad actor...,0,2021-06-28,NaN,h3aay2g,0,1,t1_h39w2aq,0,NaN,1970-01-01,1,Bitcoin,Not Set,5206


In [9]:
dfjunesample[dfjunesample["sentiment"] == "Not Set"]

3793    5526
3794      11
3795     286
3796      73
3797       9
        ... 
4945     264
4946     280
4947      67
4948      44
4949      50
Name: body, Length: 1157, dtype: int64

In [7]:
dfjunesample.dtypes

Unnamed: 0            int64
author_premium        int64
body                 object
controversiality      int64
created_utc          object
distinguished        object
id                   object
locked                int64
no_follow             int64
parent_id            object
quarantined           int64
removal_reason      float64
retrieved_on         object
score                 int64
subreddit            object
sentiment            object
dtype: object

In [129]:
docs = dfjunesample["body"].values.tolist()

In [131]:
response = client.analyze_sentiment(documents=docs)

HttpResponseError: (InvalidDocumentBatch) Request Payload sent is too large to be processed. Limit request size to: 524288
Code: InvalidDocumentBatch
Message: Request Payload sent is too large to be processed. Limit request size to: 524288

In [130]:
docs

['&gt;  my comments in other subs appear normally.\n\nMaybe older ones (before you got shadowbanned). Anyway, good luck getting it reversed :)',
 "If you have a three year old car that's worth less than what you owe on it you did something wrong.",
 'You underestimate the gullibility and impatience of some people haha. Like you said, they either learn or keep getting rekt.',
 "I'd say there's incredibly low chance of time locked deposits not being backwards compatible, it will not be done on purpose and if there's ever a bug that passes all tests and still prevents access to time locked deposits then the bug along with the changes would just be reversed and a node hotfix version released. It shouldn't be used as emergency funds either because it's a slippery slope - when funds are available anything can become an emergency - not enough money for rent? Lets withdraw some. Car broke down? Lets withdraw some, etc. It will also teach him about investing and patience while he's watching his

In [77]:
dfjunebyday = dfjune[["created_utc","id","author_premium","controversiality", "locked", "no_follow", "score"]].groupby(["created_utc"]).agg({"id": ["count"], "author_premium": ["sum"], "controversiality": [np.mean], "locked" : ["sum"], "no_follow" : ["sum"], "score" : [np.mean]})

In [81]:
dfjunebyday

,id,author_premium,controversiality,locked,no_follow,score
,count,sum,mean,sum,sum,mean
created_utc,,,,,,
2021-06-01,5638,321,0.022880,0,4053,3.120965
2021-06-02,5153,236,0.026392,0,3606,3.104405
2021-06-03,6220,389,0.024437,0,4366,3.612540
2021-06-04,8348,333,0.025994,0,6153,3.287494
2021-06-05,9049,306,0.038899,0,6500,2.790585
2021-06-06,7899,380,0.035448,0,5543,3.322952
2021-06-07,9907,458,0.033411,2,6831,3.921066
2021-06-08,14307,499,0.030125,0,10205,2.949885


In [87]:
dfjunebyday = dfjunebyday.droplevel(axis=1, level=0).reset_index().set_axis(["Date", "CommentCount", "PremiumCount", "Controversiality", "LockedCount", "FollowCount", "Score"], axis=1, inplace=False)

In [89]:
dfjunebyday

,Date,CommentCount,PremiumCount,Controversiality,LockedCount,FollowCount,Score
0,2021-06-01,5638,321,0.022880,0,4053,3.120965
1,2021-06-02,5153,236,0.026392,0,3606,3.104405
2,2021-06-03,6220,389,0.024437,0,4366,3.612540
3,2021-06-04,8348,333,0.025994,0,6153,3.287494
4,2021-06-05,9049,306,0.038899,0,6500,2.790585
5,2021-06-06,7899,380,0.035448,0,5543,3.322952
6,2021-06-07,9907,458,0.033411,2,6831,3.921066
7,2021-06-08,14307,499,0.030125,0,10205,2.949885
8,2021-06-09,12313,476,0.018273,0,8512,3.754000
9,2021-06-10,8794,428,0.027064,3,5937,3.392882


In [45]:
pd.merge(dfjune, dfpricejune, left_on="created_utc", right_on="Date", how='outer')

,author_premium,body,controversiality,created_utc,distinguished,id,locked,no_follow,parent_id,quarantined,...,retrieved_on,score,subreddit,Date,Price,Open,High,Low,Vol.,Change %
0,False,Check this out:\n\nhttps://www.reddit.com/r/Bi...,0,2021-06-01,NaN,h04y04h,False,False,t1_h04x0ol,False,...,1970-01-01,3,Bitcoin,2021-06-01,"36,687.6","37,294.3","37,850.3","35,742.7",113.48K,-1.64%
1,False,[removed],0,2021-06-01,NaN,h04y09f,False,True,t1_h04vg47,False,...,1970-01-01,1,Bitcoin,2021-06-01,"36,687.6","37,294.3","37,850.3","35,742.7",113.48K,-1.64%
2,False,"""Alt-coin Traders vs Bitcoin HODL""\n\nNever ev...",0,2021-06-01,NaN,h04y0ux,False,True,t3_nozc2s,False,...,1970-01-01,1,Bitcoin,2021-06-01,"36,687.6","37,294.3","37,850.3","35,742.7",113.48K,-1.64%
3,False,https://twitter.com/cryptanzee/status/13994910...,0,2021-06-01,NaN,h04y35o,False,False,t3_nounpp,False,...,1970-01-01,2,Bitcoin,2021-06-01,"36,687.6","37,294.3","37,850.3","35,742.7",113.48K,-1.64%
4,False,Sorry to say mining for profit is likely not v...,0,2021-06-01,NaN,h04y5xa,False,False,t1_h04b17d,False,...,1970-01-01,1,Bitcoin,2021-06-01,"36,687.6","37,294.3","37,850.3","35,742.7",113.48K,-1.64%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226857,False,I am satoshi,0,2021-06-30,NaN,h3mftl2,False,False,t3_ob6p01,False,...,1970-01-01,14,Bitcoin,2021-06-30,"35,026.9","35,832.6","36,089.5","34,126.6",107.07K,-2.25%
226858,False,you're correct\n\ntake it from someone who alr...,0,2021-06-30,NaN,h3mfuc4,False,True,t1_h3ljz8d,False,...,1970-01-01,2,Bitcoin,2021-06-30,"35,026.9","35,832.6","36,089.5","34,126.6",107.07K,-2.25%
226859,True,"Absolutely need term limits , this is getting ...",0,2021-06-30,NaN,h3mfv2z,False,True,t1_h3jwewz,False,...,1970-01-01,1,Bitcoin,2021-06-30,"35,026.9","35,832.6","36,089.5","34,126.6",107.07K,-2.25%
226860,False,"If the politicians were smarter and younger, t...",0,2021-06-30,NaN,h3mfvqx,False,False,t3_oania1,False,...,1970-01-01,2,Bitcoin,2021-06-30,"35,026.9","35,832.6","36,089.5","34,126.6",107.07K,-2.25%


In [6]:
key = "9fbe75db05be4f4ebdfb176417a72bce"

In [7]:
endpoint = "https://masterssentiment.cognitiveservices.azure.com/"

In [8]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [19]:
df.head(10)[["body"]].values

array([['I understand what you\'re saying. Perhaps I should say "static single-key paper wallets are obsolete. Mnemonic phrase paper (or steel) wallets are the way to backup your keys"'],
       ['[deleted]'],
       ['They have electrum on iOS too\nEdit : they don’t :('],
       ['Xhej'],
       ['[removed]'],
       ["look at this guy, being invited to a new year's eve party"],
       ['[deleted]'],
       ['you know what gives you an even bigger runway? keeping your job in the meantime'],
       ['Oh cool, thanks!'],
       ['Awesome : )']], dtype=object)

In [41]:
docs = df["body"].head(10).values.tolist()

In [42]:
docs

['I understand what you\'re saying. Perhaps I should say "static single-key paper wallets are obsolete. Mnemonic phrase paper (or steel) wallets are the way to backup your keys"',
 '[deleted]',
 'They have electrum on iOS too\nEdit : they don’t :(',
 'Xhej',
 '[removed]',
 "look at this guy, being invited to a new year's eve party",
 '[deleted]',
 'you know what gives you an even bigger runway? keeping your job in the meantime',
 'Oh cool, thanks!',
 'Awesome : )']

In [24]:
docs = ['test 1', 'test 2']

In [45]:
response = client.analyze_sentiment(documents=docs)

In [47]:
for r in response:
    print("Document Sentiment: {}".format(r.sentiment))

Document Sentiment: negative
Document Sentiment: neutral
Document Sentiment: negative
Document Sentiment: neutral
Document Sentiment: neutral
Document Sentiment: neutral
Document Sentiment: neutral
Document Sentiment: positive
Document Sentiment: positive
Document Sentiment: positive


In [44]:
print("Document Sentiment: {}".format(response.sentiment))

Document Sentiment: negative


In [14]:
print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
        response.confidence_scores.positive,
        response.confidence_scores.neutral,
        response.confidence_scores.negative,
    ))

Overall scores: positive=0.02; neutral=0.97; negative=0.01 

